### About this notebook
Make sure Pump data from Armstrong loaded into the system (included in hpu data)

In this notebook you will create a new score type `PUMP HYDRAULIC EFFICIENCY`, a scoring group for PUMP asset, and use formula and work order data to calculate the `Pump Hydraulic Efficiency` score.

##### Pump Hydraulic Efficiency formula details  
Step 1:  
Apparent Power (kVA) = √3 x Voltage (V) x Current (A)  
Electric Power Consumed = Motor（Apparent Power） kVA x Power Factor (e.g. 0.75)  
Motor Shaft Power (kW) = Electrical Power Consumed (kW) x Motor Efficiency η (e.g. 0.8) x Drive Efficiency η (e.g. 0.9) /1000  

Step 2:  
Pump Hydraulic Power Output (kW) = Flow Rate x Acceleration due to Gravity, （g x Flow Rate = Total Head / 3600）  
  Example: = (28.835x0.3) x  9.8 x (62.898* 0.0037 *60)/ 3600  

Pump Hydraulic Efficiency (%) = Pump Hydraulic Power Output (kW) x 100 / Pump Input（Motor）Shaft Power (kW)  





In [ ]:
!pip3 install requests_futures
import logging
import logging.config
import os
import re
import random
import requests
import requests_futures
from requests_futures.sessions import FuturesSession
import numpy as np
import pandas as pd
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

logger = logging.getLogger(__name__)

### Update variables
1. Update `site_id`,prefix, if data is in a different organization and site.   
2. HPU_Envs.json to store the environment varaibles, upload HPU_envs.json into ws project data assets. The format of the HPU_Envs.json is the following:
```
{
   "Instruction":{
      "MX_BASE_URL":"Extract health or manage host, and replace ******* in https://*******/maximo/, e.g Get main.manage.ivt13rel88.ivt.suite.maximo.com from https://main.manage.ivt13rel88.ivt.suite.maximo.com/maximo/oslc/graphite/relengineer/index.html",
      "MX_APIKEY":"Application Administration -> Go To Administration -> Integration -> API Keys -> Copy key from admin user card (e.g. 6805t46gn3tef37pu0picpg9vcq3hsmamm1enc43), or Add API key if not exist."
   },
   "MX_APIKEY":"**************",
   "MX_BASE_URL":"https://*******/maximo/"
}
```

3. Update `model_version` based on HPU version. In MAS8.7, it's 4.0.0, in MAS8.8, it's 5.0.0, in MAS8.9, it's 6.0.0   
4. Optional:Update ASSET_CLASS if needed based on the data loaded in 3.a.    
E.g ASSET_CLASS has thress samples, when only load data for Substation Transformer, use first ASSET_CLASS sample, when using the demo data set, use second ASSET_CLASS sample. 

         

In [ ]:
global MX_BASE_URL,MX_API_CONTEXT,MX_USER,MX_PWD,MX_APIKEY,APIKEY

import json
# Opening JSON file
f = open('/project_data/data_asset/HPU_Envs.json',)
data = json.load(f)
f.close()
os.environ['MX_BASE_URL'] = data['MX_BASE_URL']
os.environ['MX_APIKEY'] = data['MX_APIKEY']

site_id="EUDEMO"
prefix="EUORG1" ##Use ORGID as prefix by default

ASSET_CLASS = [
  {
    "AssetType": "PUMP",
    "ShortName": "PUMP",
    "ModelName": None
  }]
# print(ASSET_CLASS)

In [ ]:
def _call_maximo(url_path, method, json=None, headers=None, **kwargs):
    global MX_USER 
    global MX_PWD
    global MX_APIKEY
    
    MX_APIKEY = os.environ['MX_APIKEY']

    url=""
    baseurl = os.environ['MX_BASE_URL']
   
    if headers is None:
        headers = {}
    
    if MX_APIKEY is None:
        api_context = 'oslc'
#         print(baseurl)
        url = '%s%s%s&%s&%s&%s' % (baseurl, api_context, url_path,'_lid='+MX_USER,'_lpwd='+MX_PWD,'lean=1')
    else:
        api_context = 'api'
        url = '%s%s%s' % (baseurl, api_context, url_path)
        if MX_APIKEY is not None:
            headers['apikey'] = MX_APIKEY

    ssl_verify = False
    kwargs['ssl_verify'] = ssl_verify
    
    print(url)
#     print(headers)
    print(json)
    return _call(url=url, method=method, json=json, headers=headers, timeout=30000, **kwargs)

def _call(url, method, json=None, headers=None, **kwargs):
    return api_request(url, method=method, headers=headers, json=json, **kwargs)

def api_request(url, method='get', headers=None, json=None, timeout=None, ssl_verify=True, session=None, **kwargs):
    ssl_verify = False
    timeout=300
    logger.info('method=%s, url=%s, headers=%s, timeout=%s, ssl_verify=%s, json=%s, session=%s, kwargs=%s' % (method, url, str(headers), timeout, ssl_verify, str(json), str(session), str(kwargs)))

    r = requests if session is None else session

    if method == 'get':
        resp = r.get(url, headers=headers, verify=ssl_verify, json=json, timeout=timeout, **kwargs)
    elif method == 'post':
        resp = r.post(url, headers=headers, verify=ssl_verify, json=json, timeout=timeout, **kwargs)
    elif method == 'put':
        resp = r.put(url, headers=headers, verify=ssl_verify, json=json, timeout=timeout, **kwargs)
    elif method == 'patch':
        resp = r.patch(url, headers=headers, verify=ssl_verify, json=json, timeout=timeout, **kwargs)
    elif method == 'delete':
        resp = r.delete(url, headers=headers, verify=ssl_verify, json=json, timeout=timeout, **kwargs)
    else:
        raise ValueError('unsupported_method=%s' % method)

    if isinstance(r, FuturesSession):
        return resp

    logger.info('resp.status_code=%s, method=%s, url=%s' % (str(resp.status_code), method, url))

    if resp.status_code not in (requests.codes.ok, requests.codes.created, requests.codes.accepted, requests.codes.no_content):
        logger.warning('error api request: url=%s, method=%s, status_code=%s, response_text=%s' % (url, method, str(resp.status_code), str(resp.text)))
        return None

    return resp


In [ ]:
#Create a new score type
def create_new_score_type(title,description):
    asset_score_type_payload = {
        "maxtype": "DECIMAL",
        "objectname": "ASSET",
        "description": description,
        "title": title,
        "ahblthreshold": [
            {
                "lower": 0,
                "upper": 100,
                "lowerneedsattention": True,
                "ahblthresholdrange": [
                    {
                        "color": "#24a148",
                        "color_description": "Green 50",
                        "symbol": "carbon:checkmark--filled",
                        "symbol_description": "Checkmark filled",
                        "label": "Good",
                        "title": "Good",
                        "rangename": 401,
                        "lower": 70,
                        "upper": 100
                    },
                    {
                        "color": "#F1C21B",
                        "color_description": "Yellow 30",
                        "symbol": "carbon:warning--alt--filled",
                        "symbol_description": "Warning alt filled",
                        "label": "Fair",
                        "title": "Fair",
                        "rangename": 227,
                        "lower": 40,
                        "upper": 70
                    },
                    {
                        "color": "#DA1E28",
                        "color_description": "Red 60",
                        "symbol": "carbon:close--filled",
                        "symbol_description": "Close filled",
                        "label": "Poor",
                        "title": "Poor",
                        "rangename": 551,
                        "lower": 0,
                        "upper": 40
                    }
                ]
            }
        ]
    }
    headers = {"properties": "maxtype,objectname,description,attrname,title,href,ahblthreshold,sequence"}

    resp = _call_maximo(url_path='/os/mxapiapmobjectattr?lean=1', method='post',headers=headers,json=asset_score_type_payload)
    scoreType = resp.json();  
    # print(scoreType)
    if resp.status_code in (200,201):
        attrname = scoreType.get('attrname',None)
        sequence = scoreType.get('sequence',None)

        location_score_type={
            "maxtype": "DECIMAL",
            "objectname": "LOCATIONS",
            "description": description,
            "title": title,
            "attrname": attrname,
            "sequence": sequence
        }
        resp = _call_maximo(url_path='/os/mxapiapmobjectattr?lean=1', method='post',headers=headers,json=location_score_type)
        if resp.status_code in (200,201):
            scoreType = resp.json();  
            print(scoreType)
            return attrname
        else:
            raise Exception(resp.json())

    else:
        raise Exception(resp.json())

def getScoreInfo(title):
    resp = _call_maximo(url_path='/os/mxapiapmobjectattr?lean=1&oslc.where=title="'+title+'"&oslc.select=*', method='get')
    # print(resp.json())
    if resp.status_code in (200,201):
        return resp.json()['member'][0]
    else:
        raise Exception(resp.json())


#Create query
def create_query(clausename,targetObject,oslcWhereStatement):
    objectStructure = "MXAPIASSET" if targetObject=="ASSET" else "MXAPIOPERLOC"    
    querypayload={
        "ispublic":True,
        "clausename":clausename
    }
    resp = _call_maximo(url_path='/os/'+objectStructure+'?lean=1&oslc.where='+oslcWhereStatement+'&action=savethisquery', method='post',json=querypayload)
    print(resp)

#Create asset group
def create_internal_asset_group(assetGroupLabel,clausename,targetObject):
    objectStructure = "MXAPIASSET" if targetObject=="ASSET" else "MXAPIOPERLOC"    
    
    payload={
        "usewith":targetObject,
        "intobjectname":objectStructure,
        "egtype":"QUERY",
        "clausename":clausename,
        "usedby":"SCORE",
        "grouplabel":assetGroupLabel,
        "groupcalctype":"INTERNAL"
    }

    headers = {"properties": "expgroupid,grouplabel,expgroupname,description,clausename,egtype,usewith,usedby,groupcalctype"}

    resp = _call_maximo(url_path='/os/MXAPIEXPGROUP?lean=1', method='post',headers=headers,json=payload)
    print(resp.json())
    return resp.json()
    
def add_score_into_sg(scoretype,groupname,objectname):
    payload={
        "methodname": groupname+scoretype,
        "description": "Overall condition",
        "usewith": objectname,
        "autoconversion": True,
        "isattribute": False,
        "scoretype": scoretype,
        "methodattrmap": [
            {
                "attrname": scoretype,
                "groupname": groupname
            }
        ]
    }
    headers = {"properties": "methodname,apmscoreobjectattr.title,description,active"}

    resp = _call_maximo(url_path='/os/mxapiahmethodology?lean=1', method='post',headers=headers,json=payload)
    print(resp.json())

    return resp.json()
    
def create_factorlib(name,usewith,formula,best,worst):
    payload={
        "name": name,
        "usewith": usewith,
        "curvalformula": formula,
        "factorbestscore": best,
        "factorworstscore": worst,
        "usecm": False,
        "factortype": "FORMULA"
    }
    headers = {"properties": "ahfactorlibid,factorid,description,name,curvalformula"}
    resp = _call_maximo(url_path='/os/mxapiahfactorlib?lean=1', method='post',headers=headers,json=payload)
    if resp.status_code in (200,201):
        return resp.json()
    else:
        raise Exception(resp.json())

def getFactorId(name):
    resp = _call_maximo(url_path='/os/mxapiahfactorlib?lean=1&oslc.where=name="'+name+'"&oslc.select=factorid', method='get')
    print(resp.json())
    if resp.status_code in (200,201):
        return resp.json()['member'][0]['factorid']
    else:
        raise Exception(resp.json())


def create_factors_driverlib_in_group(name,scoretype,groupname,objectname,driverbest,driverworst,factorid):
    payload={
        "name": name,
        "curvalformula": "0",
        "usewith": objectname,
        "drivertype": "FACTORS",
        "bestscore": driverbest,
        "worstscore": driverworst,
        "autoconversion": True,
        "ahmethoddriver": {
            "weight": 0,
            "methodname": groupname+scoretype
        },
        "ahdriverfactor": [
            {
                "factorid": factorid,
                "weight": 1,
                "weightpct": 100
            }
        ]
    }   

    headers = {"properties": "driverid,name,description,rel.ahdriverfactor{driverid,name,factorid,weight,weightpct,ahfactorlibdirectinfo.name,rel.ahfactorlib{name, factorid,factortype, description}}"}
    resp = _call_maximo(url_path='/os/mxapiahdriverlib?lean=1', method='post',headers=headers,json=payload)

    if resp.status_code in (200,201):
        print(resp.json())
        return resp.json()
    else:
        raise Exception(resp.json())

def create_formula_driverlib_in_group(name,formula,scoretype,groupname,objectname,driverbest,driverworst):
    payload={
        "name": name,
        "curvalformula": formula, #"count$openwo",
        "usewith": objectname,
        "drivertype": "FORMULA",
        "bestscore": driverbest,
        "worstscore": driverworst,
        "autoconversion": True,
        "ahmethoddriver": {
            "weight": 0,
            "methodname": groupname+scoretype
        }
    }

    headers = {"properties": "driverid,name,description,rel.ahdriverfactor{driverid,name,factorid,weight,weightpct,ahfactorlibdirectinfo.name,rel.ahfactorlib{name, factorid,factortype, description}}"}
    resp = _call_maximo(url_path='/os/mxapiahdriverlib?lean=1', method='post',headers=headers,json=payload)

    if resp.status_code in (200,201):
        print(resp.json())
        return resp.json()
    else:
        raise Exception(resp.json())

def create_contributor_group(name,scoretype,groupname):
    payload={
        "name": name,
        "curvalformula": "0",
        "usewith": "ASSET",
        "drivertype": "FACTORS",
        "bestscore": 100,
        "worstscore": 0,
        "autoconversion": True,
        "ahmethoddriver": {
            "weight": 0,
            "methodname": groupname+scoretype
        },
        "ahdriverfactor": "nothing"
    }
    headers = {"properties": "driverid,name,description,rel.ahdriverfactor{driverid,name,factorid,weight,weightpct,ahfactorlibdirectinfo.name,rel.ahfactorlib{name, factorid,factortype, description}}"}
    resp = _call_maximo(url_path='/os/mxapiahdriverlib?lean=1', method='post',headers=headers,json=payload)

    if resp.status_code in (200,201):
        print(resp.json())
        return resp.json()
    else:
        raise Exception(resp.json())

def add_factor_into_a_group(driverRestId,driverId,factorId):
    payload={
        "href": "oslc/os/mxapiahdriverlib/"+driverRestId,
        "ahdriverfactor": [
            {
                "_action": "Add",
                "driverid": driverId,
                "factorid": factorId
            }
        ]
    }
    headers = {
        "properties": "driverid,name,description,rel.ahdriverfactor{driverid,name,factorid,weight,weightpct,ahfactorlibdirectinfo.name,rel.ahfactorlib{name, factorid,factortype, description}}",
        "patchtype": "MERGE",
        "X-method-override":"PATCH"}

    resp = _call_maximo(url_path='/os/mxapiahdriverlib/'+driverRestId+'?lean=1', method='post',headers=headers,json=payload)

    if resp.status_code in (200,201):
        print(resp.json())
        return resp.json()
    else:
        raise Exception(resp.json())


def update_driver_weight_in_group(restId,weightList):
    ahmethoddriver = []
    for item in weightList:
        obj={}
        obj["driverid"]=item["driverid"]
        obj["weightpct"]=item["weight"]
        obj["_bulkid"]=item["driverid"]
        obj["_action"]='update'
        ahmethoddriver.append(obj)

    payload={
        "ahmethoddriver": ahmethoddriver
    }

    print(payload)
    headers = {"properties": "ahmethodologyid",
            "patchtype": "MERGE",
            "X-method-override":"PATCH"}

    resp = _call_maximo(url_path='/os/mxapiahmethodology/'+restId+'?lean=1', method='post',headers=headers,json=payload)
    
    if resp.status_code in (200,201):
        print(resp.json())
        return resp.json()
    else:
        raise Exception(resp.json())

def update_factor_weight_in_driver_group(driverRestId,weightList):
    ahdriverfactor = []
    for item in weightList:
        obj={}
        obj["factorid"]=item["factorid"]
        obj["weightpct"]=item["weight"]
        obj["zerooverride"]=item["zerooverride"]
        obj["_bulkid"]=item["factorid"]
        obj["_action"]='update'
        ahdriverfactor.append(obj)

    payload={
        "ahdriverfactor": ahdriverfactor
    }

    headers = {"properties": "ahdriverlibid,name,description,factortotalweight,rel.ahdriverfactor{driverid,name,factorid,weight,weightpct,ahfactorlibdirectinfo.name,rel.ahfactorlib{name, factorid,factortype, description}}",
            "patchtype": "MERGE",
            "X-method-override":"PATCH"}

    resp = _call_maximo(url_path='/os/mxapiahdriverlib/'+driverRestId+'?lean=1', method='post',headers=headers,json=payload)
    
    if resp.status_code in (200,201):
        print(resp.json())
        return resp.json()
    else:
        raise Exception(resp.json())


def update_wo(assetnum,prefix,action):
    headers={}
    headers['X-method-override']="BULK"
    if action=='AddChange':
        headers['patchtype']="merge"
        payload=[{        
            "assetnum": assetnum,
            "siteid":site_id,
            "wonum":prefix+assetnum,
            "wpmaterial":[
                {
                    "unitcost": random.choice([10,11,12,13,14,15,16,17,18,19,20,50]),
                    "itemqty": random.choice([100,110,120,130,140,150,160,170,180,190,200,500]),
                    "linetype": "MATERIAL",
                    "linetype_description": "Material",
                    "unitcosthaschanged": True,
                    "description": "Material",
                    "_action":"AddChange"
                }
            ],
            "_action":action #"AddChange"        
        }]
        resp = _call_maximo(url_path='/os/MXAPIWODETAIL?lean=1', method='post',json=payload,headers=headers)
        if resp.status_code in (200,201):
            print(resp.json())
            return resp.json()
        else:
            raise Exception(resp.json())
    elif action=='DELETE':
        resp = _call_maximo(url_path='/os/MXAPIWODETAIL?oslc.where=wonum="'+prefix+assetnum+'" and siteid="'+site_id+'"&lean=1', method='get',json={},headers={})
        if resp.status_code in (200,201) and len(resp.json()['member'])==1:
            print(resp.json())
            payload=[{'_meta': {
                    'uri': resp.json()['member'][0]['href'],
                    'method': 'DELETE'
            }}]
            resp = _call_maximo(url_path='/os/MXAPIWODETAIL?lean=1', method='post',json=payload,headers=headers)
            print(resp.json())
        else:
            print(f"Didn't find {prefix+assetnum} work order.")

            
#Get scores list and active scores
def getScoresForTheGroup(group_restid):   
    resp = _call_maximo(url_path='/os/MXAPIEXPGROUP/'+group_restid+'/ahmethodology?oslc.select=scoretype%2Cahmethodologyid%2Capmscoreobjectattr.title%2Cdescription%2Cactive%2Cscoredependencies&oslc.pageSize=20&oslc.orderBy=%2Bahmethodologyid&searchAttributes=description&collectioncount=1&ignorecollectionref=1&relativeuri=1&addschema=1&lean=1&internalvalues=1', method='get')
#     print(resp.json())
    return resp.json()

def getScoreRestId(ahmethodologyid):   
    resp = _call_maximo(url_path='/os/mxapiahmethodology?lean=1&oslc.select=href,ahmethodologyid&oslc.where=ahmethodologyid='+str(ahmethodologyid)+'', method='get')
    data = resp.json()
#     print(data['member'][0]['href'].split("/")[-1:][0])
    return data['member'][0]['href'].split("/")[-1:][0]

def activeScore(score_restid,active):
    payload={
        "active":active
    }
    headers = {"patchtype": "MERGE"}
    headers ["x-method-override"] = "PATCH"
    headers ["properties"] = "grouplabel,active,scoretype"

    resp = _call_maximo(url_path='/os/mxapiahmethodology/'+score_restid+'?lean=1', method='post',headers=headers,json=payload)
    if resp.status_code in (200,201):
        # print(resp.json())
        return resp.json()
    else:
        raise Exception(resp.json())


# #####For clean up#####
def get_asset_group_info(grouplabel):
    if grouplabel is None:
        raise ValueError('the given argument grouplabel is None')
    resp = _call_maximo(url_path='/os/MXAPIEXPGROUP?lean=1&addid=1&oslc.where=grouplabel=%22'+grouplabel+'%22&oslc.select=*', method='get')
    if resp is not None:
        data = resp.json()
#         print(data)
        if len(data['member']) != 0:
            return (data['member'][0])
    return None

def delete_asset_group(grouplabel):
    if grouplabel is None:
        raise ValueError('the given argument grouplabel is None')
    group = get_asset_group_info(grouplabel)
    if group is not None:
        scores = getScoresForTheGroup(group['href'].split("/")[-1:][0])['member']
        scores.reverse()
        print(scores)
        for s in scores:
            ahmethodologyid = s['ahmethodologyid']
            #get score restid
            scoreRestid=getScoreRestId(ahmethodologyid)

            #active score descend
            activeScore(scoreRestid,False)            
        href = group['href']
        url_path=href.split("/api")[1]
        print(url_path)
        resp = _call_maximo(url_path=url_path, method='delete')
        print(resp)

In [ ]:
for obj in ['ASSET']:
    for item in ASSET_CLASS:  
        print('\nCreate query for:')
        print(item)

        if 'ShortName' in item:
    #         classstructure.classificationid="METALSUPPORTSTRUCTURE" and siteid="TDIST"
            clausename = prefix+"_"+site_id+"_"+item["ShortName"]
            whereStatement = 'classstructure.classificationid=%22'+item['AssetType']+'%22%20and%20siteid=%22'+site_id+'%22'

            create_query(clausename,obj,whereStatement)               
        

In [ ]:
# Create PUMP HYDRAULIC EFFICIENCY score, ignore the error if it's already existed
create_new_score_type('PUMP HYDRAULIC EFFICIENCY','PUMP HYDRAULIC EFFICIENCY ')


In [ ]:
# Create PUMP HYDRAULIC EFFICIENCY factor, ignore the error if it's already existed

pe_formula = '((METERVAL("P-HEAD") * 0.3 * 9.8 * METERVAL("P-FLOW") * 0.0037 * 60) * 100) / (((METERVAL("P-VOLTAGE") * METERVAL("P-CURRENT") * 1.732050807569 * 0.75 * 0.8 * 0.9)/1000)*3600)'
pe_formula_name = 'PUMP HYDRAULIC EFFICIENCY'

create_factorlib(pe_formula_name,'ASSET',pe_formula,100,0)


In [ ]:
#Create work order
update_wo('PUMP_04714B603408','WO001_','DELETE')
update_wo('PUMP_04714B603408','WO001_','AddChange')

In [ ]:
#Delete group
for obj in ['ASSET']:
    for item in ASSET_CLASS:  
        print(item)
        if 'ShortName' in item and 'ModelName' in item:              
            group = prefix+"_"+site_id+"_"+item["ShortName"]+"_"+obj[0]
            print('\nDelete score group for:'+group)

            delete_asset_group(group)

for obj in ['ASSET']:
    for item in ASSET_CLASS:  
        print('\nCreate score group for:')
        print(item)
        if 'ShortName' in item and 'ModelName' in item:    
            clausename = prefix+"_"+site_id+"_"+item["ShortName"]
            group = clausename+"_"+obj[0]

            #Create group
            scObj = create_internal_asset_group(group,clausename,obj)
            href = scObj['href'].split("/");
            print(href[-1:])
            group_restid = href[-1:][0]
            # print(scObj)
            expgroupname = scObj['expgroupname']

            #===============Add Health Score===============
            methodologyInfo=add_score_into_sg('HEALTH',expgroupname,obj)
            # print(methodologyInfo)
            href = methodologyInfo['href'].split("/");
            # print(href[-1:])
            methodology_restid = href[-1:][0]
            #Add contributor FACRULYEARSSAMPLE
            factorid=getFactorId("FACRULYEARSSAMPLE")
            print(factorid)
            driverInfo1=create_factors_driverlib_in_group('FACRULYEARSSAMPLE','HEALTH',expgroupname,obj,0,100,factorid)

            #create a controbutor group "meter group", then add factor "PUMP HYDRAULIC EFFICIENCY"
            factorid=getFactorId("PUMP HYDRAULIC EFFICIENCY")
            driverInfo2=create_contributor_group("meter group",'HEALTH',expgroupname)
            driverHref = driverInfo2['href'].split("/");
            print(driverHref[-1:])
            driver_restid = driverHref[-1:][0]
            add_factor_into_a_group(driver_restid,driverInfo2['driverid'],factorid)
            temp=[]
            tempObj={
                'factorid':factorid,
                'weight':100,
                'zerooverride':False
            }            
            temp.append(tempObj)
            update_factor_weight_in_driver_group(driver_restid,temp)
            
            #create wo contributor
            driverInfo3=create_formula_driverlib_in_group("Open work order","count$openwo",'HEALTH',expgroupname,obj,0,2)

            #update contributors group weight at score level
            drivers=[
                {"driverid":driverInfo1['driverid'],
                "weight":25},
                {"driverid":driverInfo2['driverid'],
                "weight":50},
                {"driverid":driverInfo3['driverid'],
                "weight":25}
            ]
            update_driver_weight_in_group(methodology_restid,drivers)

            #==========Add PUMP HYDRAULIC EFFICIENCY score===============
            pumpScore = getScoreInfo('PUMP HYDRAULIC EFFICIENCY')
            methodologyInfo=add_score_into_sg(pumpScore['attrname'],expgroupname,obj)
            # print(methodologyInfo)
            href = methodologyInfo['href'].split("/");
            # print(href[-1:])
            methodology_restid = href[-1:][0]
            #Add contributor in PUMP HYDRAULIC EFFICIENCY score
            # print(factorid)
            driverInfo4=create_factors_driverlib_in_group('PUMP HYDRAULIC EFFICIENCY',pumpScore['attrname'],expgroupname,obj,0,100,factorid)
            # update driver(factortype) weight
            drivers=[{
                'driverid':driverInfo4['driverid'],
                'weight':100
            }]
            update_driver_weight_in_group(methodology_restid,drivers)

            # Active all scores
            scores = getScoresForTheGroup(group_restid)['member']
            for s in scores:
                ahmethodologyid = s['ahmethodologyid']
                print(s['ahmethodologyid'])
                #get score restid
                scoreRestid=getScoreRestId(ahmethodologyid)

                #active score
                print("Start to active score...")
                activeScore(scoreRestid,True) 
            #We can't start analysis for all assets all together.
            expgroupnamePayload={
                "expGroupName": expgroupname
                }     
            resp =_call_maximo(url_path='/service/ahinsight?action=wsmethod%3ArecalculateScoringGroups', method='post',json=expgroupnamePayload)
            print(resp)           